In [ ]:
# Mini-GPT visual con Gradio en Colab

# Paso 1: Instalación
!pip install torch sentencepiece gradio --quiet

In [ ]:
# Paso 2: Importación
import torch
import torch.nn as nn
import torch.nn.functional as F
import sentencepiece as spm
import gradio as gr


In [ ]:
# Paso 3: Configuración
BLOCK_SIZE = 8
EMBED_DIM = 64
N_HEADS = 2
N_LAYERS = 2
LEARNING_RATE = 1e-3
MODEL_PREFIX = "bpe"

In [ ]:
# Paso 4: Cargar modelo de sentencepiece entrenado
sp = spm.SentencePieceProcessor(model_file=f"{MODEL_PREFIX}.model")
vocab_size = sp.get_piece_size()

In [ ]:
# Paso 5: Tokenización
encode = lambda s: sp.encode(s, out_type=int)
decode = lambda l: sp.decode(l)

In [ ]:
# Paso 6: Modelo MiniGPT
class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.query = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.value = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) / (C**0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        return wei @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, EMBED_DIM)

    def forward(self, x):
        return self.proj(torch.cat([h(x) for h in self.heads], dim=-1))

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(EMBED_DIM, 4 * EMBED_DIM),
            nn.ReLU(),
            nn.Linear(4 * EMBED_DIM, EMBED_DIM),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = MultiHeadAttention(N_HEADS, EMBED_DIM // N_HEADS)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(EMBED_DIM)
        self.ln2 = nn.LayerNorm(EMBED_DIM)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class MiniGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, EMBED_DIM)
        self.pos_emb = nn.Embedding(BLOCK_SIZE, EMBED_DIM)
        self.blocks = nn.Sequential(*[Block() for _ in range(N_LAYERS)])
        self.ln_f = nn.LayerNorm(EMBED_DIM)
        self.head = nn.Linear(EMBED_DIM, vocab_size)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        pos_emb = self.pos_emb(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        return logits

    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=10):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits = self(idx_cond)[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            topk_probs, topk_idx = torch.topk(probs, top_k)
            probs = torch.zeros_like(probs).scatter_(1, topk_idx, topk_probs)
            probs = probs / probs.sum(dim=-1, keepdim=True)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_id), dim=1)
        return idx

In [ ]:
# Paso 7: Cargar modelo entrenado
model = MiniGPT()
model.load_state_dict(torch.load("minigpt_model.pt"))
model.eval()

In [ ]:
# Paso 8: Interfaz Gradio

def generar(input_text, tokens, temp, topk):
    context = torch.tensor([encode(input_text)], dtype=torch.long)
    out = model.generate(context, max_new_tokens=tokens, temperature=temp, top_k=topk)[0].tolist()
    return decode(out)

iface = gr.Interface(
    fn=generar,
    inputs=[
        gr.Textbox(label="Texto inicial"),
        gr.Slider(5, 100, value=20, step=1, label="Cantidad de tokens a generar"),
        gr.Slider(0.1, 2.0, value=1.0, step=0.1, label="Creatividad (Temperature)"),
        gr.Slider(1, 50, value=10, step=1, label="Top-K (limita opciones por paso)")
    ],
    outputs="text",
    title="Mini-GPT Generador Poético"
)

iface.launch(share=True)